In [23]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *

In [24]:
DATA_PATH = 'march-machine-learning-mania-2023/'

### Seeds

In [25]:
df_seeds = pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv")

### Season results

Add total rebounds for each team, and for the whole game (to later calculate rebound percentages)

In [26]:
df_season_results = pd.read_csv(DATA_PATH + "MRegularSeasonDetailedResults.csv")

df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

df_season_results['WTR'] = df_season_results['WOR'] + df_season_results['WDR']
df_season_results['LTR'] = df_season_results['LOR'] + df_season_results['LDR']
df_season_results['TR'] = df_season_results['WTR'] + df_season_results['LTR']
df_season_results['ScoreGap'] = df_season_results['WScore'] - df_season_results['LScore']

df_season_results.drop(['WStl', 'WBlk', 'WPF', 'LStl', 'LBlk', 'LPF'], axis=1, inplace=True)


### Features
Aggregate the results for the fields we will later use for our features.

For each team at each season, computing:
- Number of wins
- Number of losses
- Average score gap of wins
- Average score gap of losses
- Total field goals made and attempted for wins and losses
- Total 3-point field goals made and attempted for wins and losses
- Total free throws made and attempted for wins and losses
- Total rebounds by the team for wins and losses
- Total rebounds in the game for wins and losses
- Total assists for wins and losses
- Total turnovers for wins and losses

And use the following features ([explained here](https://en.wikipedia.org/wiki/Basketball_statistics)):

- Win Ratio
- Average score gap
- Field goal %
- 3-point field goal %
- Free throw %
- [Effective field goal %](https://en.wikipedia.org/wiki/Effective_field_goal_percentage)
- [True shooting %](https://en.wikipedia.org/wiki/True_shooting_percentage)
- Total rebound %
- Assist to turnover ratio
- Turnover %
- Offensive rating (Points scored per possession)
- Defensive rating (Points allowed per opposition possession)
- [Net rating (Offensive rating - Defensive rating)](https://www.pivotanalysis.com/post/net-rating)


In [27]:
num_win = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})
num_loss = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})
gap_win = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()[['Season', 'WTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapWins", "WTeamID": "TeamID"})
gap_loss = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()[['Season', 'LTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapLosses", "LTeamID": "TeamID"})
fgm_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFGM']].rename(columns={"WFGM": "FGMWins", "WTeamID": "TeamID"})
fgm_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFGM']].rename(columns={"LFGM": "FGMLosses", "LTeamID": "TeamID"})
fga_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFGA']].rename(columns={"WFGA": "FGAWins", "WTeamID": "TeamID"})
fga_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFGA']].rename(columns={"LFGA": "FGALosses", "LTeamID": "TeamID"})
fgm3_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFGM3']].rename(columns={"WFGM3": "FGM3Wins", "WTeamID": "TeamID"})
fgm3_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFGM3']].rename(columns={"LFGM3": "FGM3Losses", "LTeamID": "TeamID"})
fga3_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFGA3']].rename(columns={"WFGA3": "FGA3Wins", "WTeamID": "TeamID"})
fga3_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFGA3']].rename(columns={"LFGA3": "FGA3Losses", "LTeamID": "TeamID"})
ftm_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFTM']].rename(columns={"WFTM": "FTMWins", "WTeamID": "TeamID"})
ftm_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFTM']].rename(columns={"LFTM": "FTMLosses", "LTeamID": "TeamID"})
fta_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFTA']].rename(columns={"WFTA": "FTAWins", "WTeamID": "TeamID"})
fta_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFTA']].rename(columns={"LFTA": "FTALosses", "LTeamID": "TeamID"})
teamreb_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WTR']].rename(columns={"WTR": "TeamRebWins", "WTeamID": "TeamID"})
teamreb_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LTR']].rename(columns={"LTR": "TeamRebLosses", "LTeamID": "TeamID"})
gamereb_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'TR']].rename(columns={"TR": "GameRebWins", "WTeamID": "TeamID"})
gamereb_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'TR']].rename(columns={"TR": "GameRebLosses", "LTeamID": "TeamID"})
ast_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WAst']].rename(columns={"WAst": "AstWins", "WTeamID": "TeamID"})
ast_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LAst']].rename(columns={"LAst": "AstLosses", "LTeamID": "TeamID"})
to_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WTO']].rename(columns={"WTO": "TOWins", "WTeamID": "TeamID"})
to_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LTO']].rename(columns={"LTO": "TOLosses", "LTeamID": "TeamID"})
opp_pts_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'LScore']].rename(columns={"LScore": "oppPTSWins", "WTeamID": "TeamID"})
opp_pts_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'WScore']].rename(columns={"WScore": "oppPTSLosses", "LTeamID": "TeamID"})
opp_fga_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'LFGA']].rename(columns={"LFGA": "oppFGAWins", "WTeamID": "TeamID"})
opp_fga_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'WFGA']].rename(columns={"WFGA": "oppFGALosses", "LTeamID": "TeamID"})
opp_fta_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'LFTA']].rename(columns={"LFTA": "oppFTAWins", "WTeamID": "TeamID"})
opp_fta_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'WFTA']].rename(columns={"WFTA": "oppFTALosses", "LTeamID": "TeamID"})
opp_to_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'LTO']].rename(columns={"LTO": "oppTOWins", "WTeamID": "TeamID"})
opp_to_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'WTO']].rename(columns={"WTO": "oppTOLosses", "LTeamID": "TeamID"})


In [28]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})
df_features_season = pd.concat([
  df_features_season_w, 
  df_features_season_l
], axis=0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

#### Merge, compute aggregates

In [29]:
df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fgm_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fgm_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fga_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fga_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fgm3_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fgm3_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fga3_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fga3_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(ftm_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(ftm_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fta_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fta_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(teamreb_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(teamreb_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gamereb_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gamereb_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(ast_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(ast_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(to_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(to_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_pts_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_pts_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_fga_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_fga_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_fta_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_fta_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_to_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_to_loss, on=['Season', 'TeamID'], how='left')

df_features_season.fillna(0, inplace=True)

df_features_season['FGM'] = df_features_season['FGMWins'] + df_features_season['FGMLosses']
df_features_season['FGA'] = df_features_season['FGAWins'] + df_features_season['FGALosses']
df_features_season['FGM3'] = df_features_season['FGM3Wins'] + df_features_season['FGM3Losses']
df_features_season['FGA3'] = df_features_season['FGA3Wins'] + df_features_season['FGA3Losses']
df_features_season['FTM'] = df_features_season['FTMWins'] + df_features_season['FTMLosses']
df_features_season['FTA'] = df_features_season['FTAWins'] + df_features_season['FTALosses']
df_features_season['TeamReb'] = df_features_season['TeamRebWins'] + df_features_season['TeamRebLosses']
df_features_season['GameReb'] = df_features_season['GameRebWins'] + df_features_season['GameRebLosses']
df_features_season['Ast'] = df_features_season['AstWins'] + df_features_season['AstLosses']
df_features_season['TO'] = df_features_season['TOWins'] + df_features_season['TOLosses']
df_features_season['PTS'] = 2 * df_features_season['FGM'] + df_features_season['FGM3'] + df_features_season['FTM']
df_features_season['POSS'] = df_features_season['FGA'] + 0.44 * df_features_season['FTA'] + df_features_season['TO']
df_features_season['oppPTS'] = df_features_season['oppPTSWins'] + df_features_season['oppPTSLosses']
df_features_season['oppFGA'] = df_features_season['oppFGAWins'] + df_features_season['oppFGALosses']
df_features_season['oppFTA'] = df_features_season['oppFTAWins'] + df_features_season['oppFTALosses']
df_features_season['oppTO'] = df_features_season['oppTOWins'] + df_features_season['oppTOLosses']
df_features_season['oppPOSS'] = df_features_season['oppFGA'] + 0.44 * df_features_season['oppFTA'] + df_features_season['oppTO']

#### Compute features

In [30]:
df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLosses'] * df_features_season['GapLosses'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)
df_features_season['FieldGoal%'] = df_features_season['FGM'] / df_features_season['FGA']
df_features_season['3PointFG%'] = df_features_season['FGM3'] / df_features_season['FGA3']
df_features_season['FreeThrow%'] = df_features_season['FTM'] / df_features_season['FTA']
df_features_season['effectiveFG%'] = (df_features_season['FGM'] + 0.5 * df_features_season['FGM3']) / df_features_season['FGA']
df_features_season['TrueShooting%'] = df_features_season['PTS'] / (2 * df_features_season['FGA'] + 0.88 * df_features_season['FTA'])
df_features_season['TotalRebound%'] = df_features_season['TeamReb'] / df_features_season['GameReb']
df_features_season['AssistToTurnover'] = df_features_season['Ast'] / df_features_season['TO']
df_features_season['Turnover%'] = df_features_season['TO'] / df_features_season['POSS']
df_features_season['OffRating'] = df_features_season['PTS'] / df_features_season['POSS'] * 100
df_features_season['DefRating'] = df_features_season['oppPTS'] / df_features_season['oppPOSS'] * 100
df_features_season['NetRating'] = df_features_season['OffRating'] - df_features_season['DefRating']

features = ['FieldGoal%', '3PointFG%', 'FreeThrow%', 'effectiveFG%', 'TrueShooting%', 'TotalRebound%', 'AssistToTurnover', 'Turnover%', 'OffRating', 'DefRating', 'NetRating']
targets = ['WinRatio', 'GapAvg']
keep = ['Season', 'TeamID'] + features + targets
df_features_season.drop([col for col in df_features_season.columns.values if col not in keep], axis=1, inplace=True)
df_features_season.head()

,Season,TeamID,WinRatio,GapAvg,FieldGoal%,3PointFG%,FreeThrow%,effectiveFG%,TrueShooting%,TotalRebound%,AssistToTurnover,Turnover%,OffRating,DefRating,NetRating
0,2003,1102,0.428571,0.250000,0.481149,0.375643,0.651357,0.579443,0.605015,0.413793,1.137500,0.194557,97.461028,89.253758,8.207270
1,2003,1103,0.481481,0.629630,0.486074,0.338710,0.736390,0.534814,0.585912,0.465738,1.205279,0.158154,98.649426,95.211451,3.437975
2,2003,1104,0.607143,4.285714,0.420362,0.320144,0.709898,0.475953,0.521831,0.527903,0.911290,0.166753,86.962758,84.525358,2.437401
3,2003,1105,0.269231,-4.884615,0.395755,0.364815,0.705986,0.457241,0.503801,0.480565,0.779381,0.207538,79.848690,86.566360,-6.717671
4,2003,1106,0.464286,-0.142857,0.423773,0.346154,0.646421,0.479005,0.508613,0.517647,0.685535,0.214109,79.942904,81.596270,-1.653366


In [31]:
def correlations(df, features, target, extra = ''):
  corrs = {}
  for feature in features + [extra]:
    if feature in df.columns:
      corrs[feature] = round(df[feature].corr(df[target]), 4)
  return dict(sorted(corrs.items(), key=lambda item: abs(item[1]), reverse=True))

Correlations with WinRatio

In [32]:
correlations(df_features_season, features, 'WinRatio')

{'NetRating': 0.9225,
 'OffRating': 0.7018,
 'FieldGoal%': 0.6592,
 'TrueShooting%': 0.6557,
 'DefRating': -0.6537,
 'effectiveFG%': 0.6315,
 'TotalRebound%': 0.6035,
 'AssistToTurnover': 0.5982,
 '3PointFG%': 0.4457,
 'Turnover%': -0.4217,
 'FreeThrow%': 0.2711}

Correlation with GapAvg (aka avg +/-)

In [33]:
correlations(df_features_season, features, 'GapAvg')

{'NetRating': 0.9635,
 'OffRating': 0.7347,
 'FieldGoal%': 0.6922,
 'TrueShooting%': 0.6807,
 'DefRating': -0.6807,
 'effectiveFG%': 0.6634,
 'TotalRebound%': 0.6486,
 'AssistToTurnover': 0.6479,
 '3PointFG%': 0.4686,
 'Turnover%': -0.4548,
 'FreeThrow%': 0.2827}